In [1]:
import pandas as pd
import numpy as np
import re

# 데이터 열기

In [2]:
df_movie = pd.read_csv('./data/x_train_valid.csv')
df_oscars = pd.read_csv('./data/oscars_2020_2025.csv')
df_tmbd_top = pd.read_csv('./data/tmbd_top10_2020_2025.csv')
df_movie_train = pd.read_csv('./data/x_train_valid.csv')

## 데이터 마지막 확인

In [3]:
print("df_movie.columns:", df_movie.columns.tolist())
print("df_oscars.columns:", df_oscars.columns.tolist())
print("df_tmbd_top.columns:", df_tmbd_top.columns.tolist())

df_movie.columns: ['title', 'release_date', 'type', 'runtime', 'season_count', 'episode_count', 'genre', 'director', 'cast', 'country', 'language', 'production_company', 'synopsis', 'tags', 'imdb_rating', 'imdb_review_count', 'text_input', 'production_company_is_missing', 'director_is_missing', 'cast_is_missing']
df_oscars.columns: ['Ceremony', 'Year', 'Class', 'Category', 'Film', 'Name', 'Winner']
df_tmbd_top.columns: ['country', 'country_name', 'title', 'release_date', 'vote_average', 'vote_count']


In [4]:
df_movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66300 entries, 0 to 66299
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   title                          66300 non-null  object 
 1   release_date                   66300 non-null  object 
 2   type                           66300 non-null  object 
 3   runtime                        66300 non-null  float64
 4   season_count                   66300 non-null  float64
 5   episode_count                  66300 non-null  float64
 6   genre                          63278 non-null  object 
 7   director                       66300 non-null  object 
 8   cast                           66300 non-null  object 
 9   country                        66300 non-null  object 
 10  language                       66300 non-null  object 
 11  production_company             66300 non-null  object 
 12  synopsis                       61682 non-null 

In [5]:
df_oscars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 770 entries, 0 to 769
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Ceremony  770 non-null    int64 
 1   Year      770 non-null    int64 
 2   Class     770 non-null    object
 3   Category  770 non-null    object
 4   Film      714 non-null    object
 5   Name      715 non-null    object
 6   Winner    195 non-null    object
dtypes: int64(2), object(5)
memory usage: 42.2+ KB


In [6]:
df_tmbd_top.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333 entries, 0 to 332
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country       333 non-null    object 
 1   country_name  333 non-null    object 
 2   title         333 non-null    object 
 3   release_date  333 non-null    object 
 4   vote_average  333 non-null    float64
 5   vote_count    333 non-null    int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 15.7+ KB


In [7]:
df_movie.describe()

,runtime,season_count,episode_count,imdb_rating,imdb_review_count,production_company_is_missing,director_is_missing,cast_is_missing
count,66300.000000,66300.000000,66300.000000,66300.000000,66300.000000,66300.000000,66300.000000,66300.000000
mean,71.011810,1.081267,5.827481,4.183419,45.610483,0.284087,0.162926,0.050015
std,56.308355,0.543860,22.268006,3.337211,344.562730,0.450982,0.369301,0.217978
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,35.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,79.000000,1.000000,1.000000,5.333000,2.000000,0.000000,0.000000,0.000000
75%,97.000000,1.000000,1.000000,7.000000,8.000000,1.000000,0.000000,0.000000
max,6250.000000,62.000000,1204.000000,10.000000,20819.000000,1.000000,1.000000,1.000000


## 나머지 결측치 전처리

###  df_movie

In [8]:
df_movie['release_date'] = pd.to_datetime(df_movie['release_date']).dt.year.astype(str)

In [9]:
df_movie[['title', 'imdb_rating', 'imdb_review_count']]

,title,imdb_rating,imdb_review_count
0,Stepmom's Desire,7.150,20
1,Pizza Dare 1,6.222,9
2,Bosomy Mom,5.900,11
3,Sonic the Hedgehog,7.307,9955
4,12,5.481,53
...,...,...,...
66295,Face à l'histoire : les Résistantes,8.000,1
66296,La Noche De Los Récords,0.000,0
66297,玄学小奶包竟能预测未来,7.000,1
66298,行至爱意消散处,7.000,1


In [10]:
df_movie[df_movie['genre'].isna()]

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,language,production_company,synopsis,tags,imdb_rating,imdb_review_count,text_input,production_company_is_missing,director_is_missing,cast_is_missing
4,12,2020,movie,15.0,1.0,1.0,NaN,Salih Hashwan,unknown,SA,ar,unknown,not there yet,"short film, short film",5.481,53,not there yet,1,0,1
628,Fight the Dragon: Big Breasts Game of Death,2020,movie,71.0,1.0,1.0,NaN,Hirohisa Sasaki,"Marina Yuzuki, Karina Nishida, Nina Nishimura,...",JP,ja,unknown,"The three sisters Satomi (Marina Yuzuki), Kaor...",short film,0.000,0,"The three sisters Satomi (Marina Yuzuki), Kaor...",1,0,0
659,From...,2020,movie,10.0,1.0,1.0,NaN,Sepideh Yadegar,unknown,US,en,unknown,This visual and musical feast sets the table f...,"romance, coming-of-age, rom-com, musical, adve...",7.500,2,This visual and musical feast sets the table f...,1,0,1
811,Zombie Wars,2020,movie,79.0,1.0,1.0,NaN,"Jason Wright, Kenneth Kjeldsen, Stefan Androv ...","Rickey Bird Jr., Laurence R. Harvey, Megan Loc...",GB,en,unknown,Anthology feature with short zombie films from...,"horror, adventure, survival",1.000,1,Anthology feature with short zombie films from...,1,0,0
815,Dancing Mita,2020,movie,104.0,1.0,1.0,NaN,Toshimitsu Iizuka,"Amane Okayama, Konatsu Kato, Yuichi Nakamura, ...",JP,ja,unknown,Mita (Amane Okayama) wanted to become a videog...,"family, psychological, superhero, teen",7.300,3,Mita (Amane Okayama) wanted to become a videog...,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66249,Victim Ya Victor,2025,tv,23.0,1.0,5.0,NaN,unknown,Rahul Dua,IN,hi,unknown,"""Victim ya Victor"" is a wild game show where f...","documentary, trending-theme",0.000,0,"""Victim ya Victor"" is a wild game show where f...",1,1,0
66251,De Grannies van Amsterdam,2025,tv,45.0,1.0,10.0,NaN,unknown,"Lorraine, Mug, Joanita, Tonneke, Jocelyn",NL,nl,unknown,In De Grannies van Amsterdam the cameras follo...,"teen, feel-good, romance, family, rom-com, sta...",0.000,0,In De Grannies van Amsterdam the cameras follo...,1,1,0
66256,Brandmerk: Kind van een Gedetineerde,2025,tv,35.0,1.0,6.0,NaN,unknown,Menen Seijkens,NL,nl,unknown,"In the Netherlands, at least 25,000 children h...",family,0.000,0,"In the Netherlands, at least 25,000 children h...",1,1,0
66258,Vicky Pattison: My Deepfake Sex Tape,2025,tv,47.0,1.0,1.0,NaN,unknown,Vicky Pattison,GB,en,unknown,Vicky boldly immerses herself in the world of ...,"historical, noir",0.000,0,Vicky boldly immerses herself in the world of ...,1,1,0


In [11]:
df_movie.head()

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,language,production_company,synopsis,tags,imdb_rating,imdb_review_count,text_input,production_company_is_missing,director_is_missing,cast_is_missing
0,Stepmom's Desire,2020,movie,78.0,1.0,1.0,"Drama, Romance",Lee Dong-Joon,"Lee Soo, Tae Hee, James, Jung In",KR,ko,unknown,"Sang-jin, who was envious of her neighbor afte...",softcore,7.150,20,"Sang-jin, who was envious of her neighbor afte...",1,0,0
1,Pizza Dare 1,2020,movie,68.0,1.0,1.0,Romance,Kong Ja-kwan,"Yoo Jung, Yoon So-jung, Jin Si-ah, Baek Yoon-j...",KR,ko,unknown,"Seok, who wants to try perverted sex, pesters ...",softcore,6.222,9,"Seok, who wants to try perverted sex, pesters ...",1,0,0
2,Bosomy Mom,2020,movie,69.0,1.0,1.0,"Drama, Romance",Lee Dong-Joon,"Yoo Jung, Han Yi-seul, James, Shin Yeon-woo",KR,ko,unknown,Da-hee gets married to Min-soo who is older th...,softcore,5.900,11,Da-hee gets married to Min-soo who is older th...,1,0,0
3,Sonic the Hedgehog,2020,movie,99.0,1.0,1.0,"Action, Science Fiction, Comedy, Family",Jeff Fowler,"Ben Schwartz, James Marsden, Tika Sumpter, Jim...",US,en,"Original Film, Blur Studio, Marza Animation Pl...","Powered with incredible speed, Sonic The Hedge...","friendship, video game, san francisco, califor...",7.307,9955,"Powered with incredible speed, Sonic The Hedge...",0,0,0
4,12,2020,movie,15.0,1.0,1.0,NaN,Salih Hashwan,unknown,SA,ar,unknown,not there yet,"short film, short film",5.481,53,not there yet,1,0,1


- genre nan 데이터 제거

In [12]:
df_movie = df_movie.dropna(subset=["genre"])

- synopsis nan은 unknown처리

In [13]:
df_movie["synopsis"] = df_movie["synopsis"].fillna("unknown")

In [14]:
df_movie.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63278 entries, 0 to 66299
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   title                          63278 non-null  object 
 1   release_date                   63278 non-null  object 
 2   type                           63278 non-null  object 
 3   runtime                        63278 non-null  float64
 4   season_count                   63278 non-null  float64
 5   episode_count                  63278 non-null  float64
 6   genre                          63278 non-null  object 
 7   director                       63278 non-null  object 
 8   cast                           63278 non-null  object 
 9   country                        63278 non-null  object 
 10  language                       63278 non-null  object 
 11  production_company             63278 non-null  object 
 12  synopsis                       63278 non-null  obje

## df_oscars

In [15]:
df_oscars.head()

,Ceremony,Year,Class,Category,Film,Name,Winner
0,93,2020,Acting,ACTOR IN A LEADING ROLE,Sound of Metal,Riz Ahmed,NaN
1,93,2020,Acting,ACTOR IN A LEADING ROLE,Ma Rainey's Black Bottom,Chadwick Boseman,NaN
2,93,2020,Acting,ACTOR IN A LEADING ROLE,The Father,Anthony Hopkins,True
3,93,2020,Acting,ACTOR IN A LEADING ROLE,Mank,Gary Oldman,NaN
4,93,2020,Acting,ACTOR IN A LEADING ROLE,Minari,Steven Yeun,NaN


In [16]:
df_oscars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 770 entries, 0 to 769
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Ceremony  770 non-null    int64 
 1   Year      770 non-null    int64 
 2   Class     770 non-null    object
 3   Category  770 non-null    object
 4   Film      714 non-null    object
 5   Name      715 non-null    object
 6   Winner    195 non-null    object
dtypes: int64(2), object(5)
memory usage: 42.2+ KB


In [17]:
df_oscars["Film"] = df_oscars["Film"].fillna("unknown")
df_oscars["Film"] = df_oscars["Film"].fillna("unknown")

In [18]:
df_oscars["Winner"] = df_oscars["Winner"].astype(str).str.lower() == "true"

In [19]:
df_oscars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 770 entries, 0 to 769
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Ceremony  770 non-null    int64 
 1   Year      770 non-null    int64 
 2   Class     770 non-null    object
 3   Category  770 non-null    object
 4   Film      770 non-null    object
 5   Name      715 non-null    object
 6   Winner    770 non-null    bool  
dtypes: bool(1), int64(2), object(4)
memory usage: 37.0+ KB


## df_tmbd_top

In [20]:
df_tmbd_top.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333 entries, 0 to 332
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country       333 non-null    object 
 1   country_name  333 non-null    object 
 2   title         333 non-null    object 
 3   release_date  333 non-null    object 
 4   vote_average  333 non-null    float64
 5   vote_count    333 non-null    int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 15.7+ KB


In [21]:
df_tmbd_top['release_date'] = pd.to_datetime(df_tmbd_top['release_date']).dt.year.astype(str)
df_tmbd_top.head()

,country,country_name,title,release_date,vote_average,vote_count
0,US,United States,Folklore: The Long Pond Studio Sessions,2020,8.500,165
1,US,United States,Gabriel's Inferno,2020,8.411,2423
2,US,United States,Gabriel's Inferno: Part II,2020,8.400,1522
3,US,United States,Gabriel's Inferno: Part III,2020,8.400,1058
4,US,United States,RBD: Ser o Parecer (En Vivo),2020,8.400,83


# df_movie

In [22]:
df_movie.describe()

,runtime,season_count,episode_count,imdb_rating,imdb_review_count,production_company_is_missing,director_is_missing,cast_is_missing
count,63278.000000,63278.000000,63278.000000,63278.000000,63278.000000,63278.000000,63278.000000,63278.000000
mean,71.535257,1.083441,5.949414,4.313007,47.763931,0.263551,0.159597,0.046746
std,50.508485,0.553268,22.614478,3.301194,352.550047,0.440562,0.366235,0.211096
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,80.000000,1.000000,1.000000,5.500000,2.000000,0.000000,0.000000,0.000000
75%,98.000000,1.000000,1.000000,7.000000,9.000000,1.000000,0.000000,0.000000
max,6250.000000,62.000000,1204.000000,10.000000,20819.000000,1.000000,1.000000,1.000000


<strong>- imdb_rating * imdb_review_count 하여 상위 20%에 1 부여</strong>

In [23]:
df_movie["imdb_score"] = df_movie["imdb_rating"] * np.log1p(df_movie["imdb_review_count"])
threshold = df_movie["imdb_score"].quantile(0.8)

In [24]:
df_movie.head()

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,...,production_company,synopsis,tags,imdb_rating,imdb_review_count,text_input,production_company_is_missing,director_is_missing,cast_is_missing,imdb_score
0,Stepmom's Desire,2020,movie,78.0,1.0,1.0,"Drama, Romance",Lee Dong-Joon,"Lee Soo, Tae Hee, James, Jung In",KR,...,unknown,"Sang-jin, who was envious of her neighbor afte...",softcore,7.150,20,"Sang-jin, who was envious of her neighbor afte...",1,0,0,21.768335
1,Pizza Dare 1,2020,movie,68.0,1.0,1.0,Romance,Kong Ja-kwan,"Yoo Jung, Yoon So-jung, Jin Si-ah, Baek Yoon-j...",KR,...,unknown,"Seok, who wants to try perverted sex, pesters ...",softcore,6.222,9,"Seok, who wants to try perverted sex, pesters ...",1,0,0,14.326684
2,Bosomy Mom,2020,movie,69.0,1.0,1.0,"Drama, Romance",Lee Dong-Joon,"Yoo Jung, Han Yi-seul, James, Shin Yeon-woo",KR,...,unknown,Da-hee gets married to Min-soo who is older th...,softcore,5.900,11,Da-hee gets married to Min-soo who is older th...,1,0,0,14.660949
3,Sonic the Hedgehog,2020,movie,99.0,1.0,1.0,"Action, Science Fiction, Comedy, Family",Jeff Fowler,"Ben Schwartz, James Marsden, Tika Sumpter, Jim...",US,...,"Original Film, Blur Studio, Marza Animation Pl...","Powered with incredible speed, Sonic The Hedge...","friendship, video game, san francisco, califor...",7.307,9955,"Powered with incredible speed, Sonic The Hedge...",0,0,0,67.267735
5,Demon Slayer -Kimetsu no Yaiba- The Movie: Mug...,2020,movie,117.0,1.0,1.0,"Animation, Action, Fantasy, Thriller",Haruo Sotozaki,"Natsuki Hanae, Akari Kito, Hiro Shimono, Yoshi...",JP,...,"ufotable, Aniplex, Shueisha, TOHO","Tanjiro Kamado, joined with Inosuke Hashibira,...","fight, magic, supernatural, psychology, gore, ...",8.209,4103,"Tanjiro Kamado, joined with Inosuke Hashibira,...",0,0,0,68.296560


In [25]:
df_movie.shape

(63278, 21)

# concat oscars


| 클래스                                                 | 의미            | 점수   |
| --------------------------------------------------- | ------------- | ---- |
| 수상 (`Winner == "True"`)                             | 실제 오스카 수상작/인물 | `2`점 |
| 후보 (`Class == "Nomination"` and `Winner != "True"`) | 수상은 못 했지만 후보작 | `1`점 |
| 없음                                                  | 오스카 언급 안 됨    | `0`점 |

In [26]:
def assign_award_score(row):
    if str(row["Winner"]).strip().lower() == "true":
        return 2
    elif str(row["Class"]).strip().lower() == "nomination":
        return 1
    else:
        return 1
    
df_oscars["award_score"] = df_oscars.apply(assign_award_score, axis=1)

In [27]:
df_oscars["award_score"].value_counts()

award_score
1    575
2    195
Name: count, dtype: int64

In [28]:
df_oscars["award_score"].value_counts(normalize=True)

award_score
1    0.746753
2    0.253247
Name: proportion, dtype: float64

In [29]:
df_oscars.head()

,Ceremony,Year,Class,Category,Film,Name,Winner,award_score
0,93,2020,Acting,ACTOR IN A LEADING ROLE,Sound of Metal,Riz Ahmed,False,1
1,93,2020,Acting,ACTOR IN A LEADING ROLE,Ma Rainey's Black Bottom,Chadwick Boseman,False,1
2,93,2020,Acting,ACTOR IN A LEADING ROLE,The Father,Anthony Hopkins,True,2
3,93,2020,Acting,ACTOR IN A LEADING ROLE,Mank,Gary Oldman,False,1
4,93,2020,Acting,ACTOR IN A LEADING ROLE,Minari,Steven Yeun,False,1


# Top 10

In [30]:
df_tmbd_top.head()

,country,country_name,title,release_date,vote_average,vote_count
0,US,United States,Folklore: The Long Pond Studio Sessions,2020,8.500,165
1,US,United States,Gabriel's Inferno,2020,8.411,2423
2,US,United States,Gabriel's Inferno: Part II,2020,8.400,1522
3,US,United States,Gabriel's Inferno: Part III,2020,8.400,1058
4,US,United States,RBD: Ser o Parecer (En Vivo),2020,8.400,83


In [31]:
# 정렬 기준이 필요해요: vote_average or vote_count 기준
df_tmbd_top = df_tmbd_top.sort_values(["country", "vote_average", "vote_count"], ascending=[True, False, False])

# 기본 점수는 1.0
df_tmbd_top["top_score"] = 1.0

# 상위 20%: 나라별 상위 2편에게 +0.1점
df_tmbd_top["bonus_score"] = (
    df_tmbd_top.groupby("country").cumcount() < 2  # 상위 0, 1번 인덱스만
).astype(float) * 0.1

# 최종 점수
df_tmbd_top["top_score"] += df_tmbd_top["bonus_score"]

In [32]:
df_tmbd_top["title_lower"] = df_tmbd_top["title"].str.lower()

top_scores = df_tmbd_top.groupby("title_lower")["top_score"].sum().reset_index()

In [33]:
df_tmbd_top = df_tmbd_top.drop(columns=["bonus_score", "title_lower"])
df_tmbd_top

,country,country_name,title,release_date,vote_average,vote_count,top_score
100,CN,China,Green Snake,2021,8.100,327,1.1
101,CN,China,New Gods: Nezha Reborn,2021,8.095,462,1.1
325,CN,China,Ne Zha 2,2025,8.042,179,1.0
220,CN,China,Hachiko,2023,7.879,253,1.0
102,CN,China,B for Busy,2021,7.700,54,1.0
...,...,...,...,...,...,...,...
306,US,United States,The Life of Chuck,2025,7.737,59,1.0
305,US,United States,The Gorge,2025,7.711,2706,1.0
307,US,United States,The Unbreakable Boy,2025,7.630,73,1.0
308,US,United States,Sinners,2025,7.549,1563,1.0


In [34]:
df_tmbd_top.columns

Index(['country', 'country_name', 'title', 'release_date', 'vote_average',
       'vote_count', 'top_score'],
      dtype='object')

In [35]:
df_tmbd_top = df_tmbd_top[['country', 'title', 'release_date', 'vote_average', 'vote_count', 'top_score']]
df_tmbd_top.head()

,country,title,release_date,vote_average,vote_count,top_score
100,CN,Green Snake,2021,8.100,327,1.1
101,CN,New Gods: Nezha Reborn,2021,8.095,462,1.1
325,CN,Ne Zha 2,2025,8.042,179,1.0
220,CN,Hachiko,2023,7.879,253,1.0
102,CN,B for Busy,2021,7.700,54,1.0


In [36]:
print(df_movie.info())
print(df_oscars.info())
print(df_tmbd_top.info())

<class 'pandas.core.frame.DataFrame'>
Index: 63278 entries, 0 to 66299
Data columns (total 21 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   title                          63278 non-null  object 
 1   release_date                   63278 non-null  object 
 2   type                           63278 non-null  object 
 3   runtime                        63278 non-null  float64
 4   season_count                   63278 non-null  float64
 5   episode_count                  63278 non-null  float64
 6   genre                          63278 non-null  object 
 7   director                       63278 non-null  object 
 8   cast                           63278 non-null  object 
 9   country                        63278 non-null  object 
 10  language                       63278 non-null  object 
 11  production_company             63278 non-null  object 
 12  synopsis                       63278 non-null  obje

In [37]:
df_oscars['Class'].value_counts(dropna=False)

Class
Title             198
Production        180
Acting            120
Writing            60
Music              55
SciTech            37
Directing          30
Special            20
Short              10
Documentary        10
Makeup              5
Sound               5
Picture             5
Song                5
Costume             5
International       5
Editing             5
Cinematography      5
Animated            5
VFX                 5
Name: count, dtype: int64

In [38]:
# 1. 날짜 타입 변환
df_oscars["Year"] = pd.to_datetime(df_oscars["Year"], errors="coerce")  # 예: "2021" → Timestamp
df_movie["release_date"] = pd.to_datetime(df_movie["release_date"], errors="coerce")

# 2. Acting 수상자만 필터
acting = df_oscars[df_oscars["Class"] == "acting"].dropna(subset=["Name", "Year"])

# 3. df_movie에 cast_list 컬럼 추가
df_movie["cast_list"] = df_movie["cast"].str.split(",")

# 4. 배우 수상 내역을 dictionary로 정리 (배우별 → 수상 연도 리스트)
from collections import defaultdict

actor_awards_by_year = defaultdict(list)
for _, row in acting.iterrows():
    name = row["Name"].strip()
    year = row["Year"]
    score = row["award_score"]
    actor_awards_by_year[name].append((year, score))

# 5. 점수 계산 함수 (조건: 수상일이 영화개봉일보다 같거나 이전일 때만)
def get_cast_award_score_conditional(cast_list, release_date):
    if not isinstance(cast_list, list) or pd.isna(release_date):
        return 0
    total = 0
    for actor in cast_list:
        actor = actor.strip()
        if actor in actor_awards_by_year:
            for award_date, score in actor_awards_by_year[actor]:
                if award_date <= release_date:
                    total += score
    return total

# 6. 적용
df_movie["award_score_cast"] = df_movie.apply(
    lambda row: get_cast_award_score_conditional(row["cast_list"], row["release_date"]), axis=1
)


In [39]:
# Acting 외 영화기반 점수 계산 (기존과 동일)
non_acting = df_oscars[df_oscars["Class"] != "acting"]
award_by_title = non_acting.groupby("Film")["award_score"].sum().reset_index()
award_by_title.columns = ["title", "award_score_title"]

df_movie = df_movie.merge(award_by_title, on="title", how="left")

# 최종 award_score 합산
df_movie["award_score"] = df_movie[["award_score_title", "award_score_cast"]].sum(axis=1).fillna(0)

# 중간 컬럼 정리 (선택)
df_movie.drop(columns=["award_score_title", "award_score_cast", "cast_list"], inplace=True)

In [40]:
df_movie.head()

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,...,synopsis,tags,imdb_rating,imdb_review_count,text_input,production_company_is_missing,director_is_missing,cast_is_missing,imdb_score,award_score
0,Stepmom's Desire,2020-01-01,movie,78.0,1.0,1.0,"Drama, Romance",Lee Dong-Joon,"Lee Soo, Tae Hee, James, Jung In",KR,...,"Sang-jin, who was envious of her neighbor afte...",softcore,7.150,20,"Sang-jin, who was envious of her neighbor afte...",1,0,0,21.768335,0.0
1,Pizza Dare 1,2020-01-01,movie,68.0,1.0,1.0,Romance,Kong Ja-kwan,"Yoo Jung, Yoon So-jung, Jin Si-ah, Baek Yoon-j...",KR,...,"Seok, who wants to try perverted sex, pesters ...",softcore,6.222,9,"Seok, who wants to try perverted sex, pesters ...",1,0,0,14.326684,0.0
2,Bosomy Mom,2020-01-01,movie,69.0,1.0,1.0,"Drama, Romance",Lee Dong-Joon,"Yoo Jung, Han Yi-seul, James, Shin Yeon-woo",KR,...,Da-hee gets married to Min-soo who is older th...,softcore,5.900,11,Da-hee gets married to Min-soo who is older th...,1,0,0,14.660949,0.0
3,Sonic the Hedgehog,2020-01-01,movie,99.0,1.0,1.0,"Action, Science Fiction, Comedy, Family",Jeff Fowler,"Ben Schwartz, James Marsden, Tika Sumpter, Jim...",US,...,"Powered with incredible speed, Sonic The Hedge...","friendship, video game, san francisco, califor...",7.307,9955,"Powered with incredible speed, Sonic The Hedge...",0,0,0,67.267735,0.0
4,Demon Slayer -Kimetsu no Yaiba- The Movie: Mug...,2020-01-01,movie,117.0,1.0,1.0,"Animation, Action, Fantasy, Thriller",Haruo Sotozaki,"Natsuki Hanae, Akari Kito, Hiro Shimono, Yoshi...",JP,...,"Tanjiro Kamado, joined with Inosuke Hashibira,...","fight, magic, supernatural, psychology, gore, ...",8.209,4103,"Tanjiro Kamado, joined with Inosuke Hashibira,...",0,0,0,68.296560,0.0


In [41]:
df_movie.shape

(63278, 22)

In [42]:
df_movie.sort_values(by='award_score', ascending=False)

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,...,synopsis,tags,imdb_rating,imdb_review_count,text_input,production_company_is_missing,director_is_missing,cast_is_missing,imdb_score,award_score
47122,The Brutalist,2024-01-01,movie,215.0,1.0,1.0,Drama,Brady Corbet,"Adrien Brody, Felicity Jones, Guy Pearce, Joe ...",US,...,When a visionary architect and his wife flee p...,"new york city, immigrant, prostitute, husband ...",7.006,1118,When a visionary architect and his wife flee p...,0,0,0,49.183456,33.0
47079,Anora,2024-01-01,movie,139.0,1.0,1.0,"Drama, Comedy, Romance",Sean Baker,"Mikey Madison, Mark Eydelshteyn, Yura Borisov,...",US,...,A young sex worker from Brooklyn gets her chan...,"new year's eve, new york city, marriage, board...",7.061,2285,A young sex worker from Brooklyn gets her chan...,0,0,0,54.613720,31.0
47254,Emilia Pérez,2024-01-01,movie,132.0,1.0,1.0,"Drama, Thriller",Jacques Audiard,"Zoe Saldaña, Karla Sofía Gascón, Selena Gomez,...",FR,...,"Rita, an underrated lawyer working for a large...","based on novel or book, musical, lawyer, drug ...",6.604,1690,"Rita, an underrated lawyer working for a large...",0,0,0,49.088030,27.0
12162,Wicked,2021-01-01,movie,114.0,1.0,1.0,"Fantasy, Romance, Drama",Taryn O'Neill,"Liam Hall, Anna Maiche, Kevin Joy, Andrew Roge...",US,...,"Ivy Morgan, a college student in New Orleans, ...",short film,7.786,98,"Ivy Morgan, a college student in New Orleans, ...",0,0,0,35.777603,26.0
47078,Wicked,2024-01-01,movie,162.0,1.0,1.0,"Drama, Romance, Fantasy",Jon M. Chu,"Cynthia Erivo, Ariana Grande, Jeff Goldblum, M...",US,...,"In the land of Oz, ostracized and misunderstoo...","witch, dancing, based on novel or book, colleg...",6.900,2185,"In the land of Oz, ostracized and misunderstoo...",0,0,0,53.059818,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21153,All About Us,2021-01-01,tv,20.0,4.0,48.0,Drama,Duc Ngo Ngoc,"Thi Le Thanh Ho, Katharina Nesytowa, Eva Maria...",DE,...,Juggling a never-ending barrage of responsibil...,"superhero, historical, family, romance, twist,...",8.000,3,Juggling a never-ending barrage of responsibil...,0,0,0,11.090355,0.0
21154,The Game,2021-01-01,tv,30.0,2.0,20.0,"Drama, Comedy",unknown,"Wendy Raquel Robinson, Hosea Chanchez, Adriyan...",US,...,"Relocating from San Diego to Sin City, a mix o...","action, superhero, rom-com, comedy, romance, s...",8.400,8,"Relocating from San Diego to Sin City, a mix o...",0,1,0,18.456686,0.0
21155,My Treasure,2021-01-01,tv,45.0,1.0,35.0,"Drama, Family",unknown,"Wen Qi, Liu Mintao, Roy Chiu, Annie Liu Xin-Yo...",CN,...,It’s never easy to live a big city life. Faced...,"superhero, drama",5.300,3,It’s never easy to live a big city life. Faced...,1,1,0,7.347360,0.0
21156,Stephen,2021-01-01,tv,45.0,1.0,3.0,Drama,unknown,"Steve Coogan, Sharlene Whyte, Hugh Quarshie, J...",GB,...,Follows the ongoing struggle by Doreen and Nev...,"thriller, detective, historical, noir, crime, ...",6.800,14,Follows the ongoing struggle by Doreen and Nev...,0,1,0,18.414741,0.0


In [43]:
df_tmbd_top.head()

,country,title,release_date,vote_average,vote_count,top_score
100,CN,Green Snake,2021,8.100,327,1.1
101,CN,New Gods: Nezha Reborn,2021,8.095,462,1.1
325,CN,Ne Zha 2,2025,8.042,179,1.0
220,CN,Hachiko,2023,7.879,253,1.0
102,CN,B for Busy,2021,7.700,54,1.0


In [44]:
# 1. 필요한 컬럼만 추출
top_scores = df_tmbd_top[["title", "top_score"]].copy()

# 2. title 기준 merge
df_movie = df_movie.merge(top_scores, on="title", how="left")

# 3. 없는 경우는 0.0으로 대체
df_movie["top_score"] = df_movie["top_score"].fillna(0.0)

In [45]:
df_movie.head()

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,...,tags,imdb_rating,imdb_review_count,text_input,production_company_is_missing,director_is_missing,cast_is_missing,imdb_score,award_score,top_score
0,Stepmom's Desire,2020-01-01,movie,78.0,1.0,1.0,"Drama, Romance",Lee Dong-Joon,"Lee Soo, Tae Hee, James, Jung In",KR,...,softcore,7.150,20,"Sang-jin, who was envious of her neighbor afte...",1,0,0,21.768335,0.0,0.0
1,Pizza Dare 1,2020-01-01,movie,68.0,1.0,1.0,Romance,Kong Ja-kwan,"Yoo Jung, Yoon So-jung, Jin Si-ah, Baek Yoon-j...",KR,...,softcore,6.222,9,"Seok, who wants to try perverted sex, pesters ...",1,0,0,14.326684,0.0,0.0
2,Bosomy Mom,2020-01-01,movie,69.0,1.0,1.0,"Drama, Romance",Lee Dong-Joon,"Yoo Jung, Han Yi-seul, James, Shin Yeon-woo",KR,...,softcore,5.900,11,Da-hee gets married to Min-soo who is older th...,1,0,0,14.660949,0.0,0.0
3,Sonic the Hedgehog,2020-01-01,movie,99.0,1.0,1.0,"Action, Science Fiction, Comedy, Family",Jeff Fowler,"Ben Schwartz, James Marsden, Tika Sumpter, Jim...",US,...,"friendship, video game, san francisco, califor...",7.307,9955,"Powered with incredible speed, Sonic The Hedge...",0,0,0,67.267735,0.0,0.0
4,Demon Slayer -Kimetsu no Yaiba- The Movie: Mug...,2020-01-01,movie,117.0,1.0,1.0,"Animation, Action, Fantasy, Thriller",Haruo Sotozaki,"Natsuki Hanae, Akari Kito, Hiro Shimono, Yoshi...",JP,...,"fight, magic, supernatural, psychology, gore, ...",8.209,4103,"Tanjiro Kamado, joined with Inosuke Hashibira,...",0,0,0,68.296560,0.0,1.0


In [46]:
df_movie.shape

(63284, 23)

# 성공여부 결정

| 조건                      | 의미                  |
| ----------------------- | ------------------- |
| 1️⃣ `imdb_score` 상위 20% | 대중성과 평가에서 높은 점수     |
| 2️⃣ `award_score > 0`   | 전문가 집단의 인지도 (오스카 등) |
| 3️⃣ `top_score > 0`     | TMDb 기반 대중 인기 반영    |

- 조건 중 하나 이상 만족시 success_label = 1

In [47]:
df_movie["release_year"] = df_movie["release_date"].astype(str).str[:4].astype(int)

df_success = df_movie[["title", "release_year", "imdb_score", "award_score", "top_score"]].copy()

# 성공 조건 생성
imdb_threshold = df_success["imdb_score"].quantile(0.8)
df_success["success_imdb"] = (df_success["imdb_score"] >= imdb_threshold).astype(int)
df_success["success_award"] = (df_success["award_score"] > 0).astype(int)
df_success["success_tmdb"] = (df_success["top_score"] > 0).astype(int)

df_success["success_label"] = (
    df_success[["success_imdb", "success_award", "success_tmdb"]].sum(axis=1) >= 1
).astype(int)

# title + release_year 기준으로 병합
df_movie = df_movie.merge(
    df_success[["title", "release_year", "success_label"]],
    on=["title", "release_year"],
    how="left"
)

df_movie["success_label"] = df_movie["success_label"].fillna(0).astype(int)

In [48]:
df_movie.head()

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,...,imdb_review_count,text_input,production_company_is_missing,director_is_missing,cast_is_missing,imdb_score,award_score,top_score,release_year,success_label
0,Stepmom's Desire,2020-01-01,movie,78.0,1.0,1.0,"Drama, Romance",Lee Dong-Joon,"Lee Soo, Tae Hee, James, Jung In",KR,...,20,"Sang-jin, who was envious of her neighbor afte...",1,0,0,21.768335,0.0,0.0,2020,1
1,Pizza Dare 1,2020-01-01,movie,68.0,1.0,1.0,Romance,Kong Ja-kwan,"Yoo Jung, Yoon So-jung, Jin Si-ah, Baek Yoon-j...",KR,...,9,"Seok, who wants to try perverted sex, pesters ...",1,0,0,14.326684,0.0,0.0,2020,0
2,Bosomy Mom,2020-01-01,movie,69.0,1.0,1.0,"Drama, Romance",Lee Dong-Joon,"Yoo Jung, Han Yi-seul, James, Shin Yeon-woo",KR,...,11,Da-hee gets married to Min-soo who is older th...,1,0,0,14.660949,0.0,0.0,2020,0
3,Sonic the Hedgehog,2020-01-01,movie,99.0,1.0,1.0,"Action, Science Fiction, Comedy, Family",Jeff Fowler,"Ben Schwartz, James Marsden, Tika Sumpter, Jim...",US,...,9955,"Powered with incredible speed, Sonic The Hedge...",0,0,0,67.267735,0.0,0.0,2020,1
4,Demon Slayer -Kimetsu no Yaiba- The Movie: Mug...,2020-01-01,movie,117.0,1.0,1.0,"Animation, Action, Fantasy, Thriller",Haruo Sotozaki,"Natsuki Hanae, Akari Kito, Hiro Shimono, Yoshi...",JP,...,4103,"Tanjiro Kamado, joined with Inosuke Hashibira,...",0,0,0,68.296560,0.0,1.0,2020,1


In [49]:
# 0. release_date에서 연도 추출 (merge 전에 추가)
df_movie_train["year"] = pd.to_datetime(df_movie_train["release_date"], errors='coerce').dt.year
df_movie_train["year"] = df_movie_train["year"].astype('Int64')  # 결측치 고려

# 1. release_date에서 월 추출
df_movie_train["month"] = pd.to_datetime(df_movie_train["release_date"], errors='coerce').dt.month

# 2. 월을 2자리 문자열로 변환 (01, 02, ..., 12)
df_movie_train["month"] = df_movie_train["month"].apply(lambda x: f"{int(x):02d}" if pd.notnull(x) else None)

# 3. title과 synopsis가 같은 경우만 추출
df_month_map = df_movie_train[["title", "runtime", "month", "year"]].dropna()

# 4. df_movie에 merge
df_movie = df_movie.merge(
    df_month_map,
    on=["title", "runtime"],
    how="left"
)
df_movie.head()

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,...,production_company_is_missing,director_is_missing,cast_is_missing,imdb_score,award_score,top_score,release_year,success_label,month,year
0,Stepmom's Desire,2020-01-01,movie,78.0,1.0,1.0,"Drama, Romance",Lee Dong-Joon,"Lee Soo, Tae Hee, James, Jung In",KR,...,1,0,0,21.768335,0.0,0.0,2020,1,05,2020
1,Pizza Dare 1,2020-01-01,movie,68.0,1.0,1.0,Romance,Kong Ja-kwan,"Yoo Jung, Yoon So-jung, Jin Si-ah, Baek Yoon-j...",KR,...,1,0,0,14.326684,0.0,0.0,2020,0,04,2020
2,Bosomy Mom,2020-01-01,movie,69.0,1.0,1.0,"Drama, Romance",Lee Dong-Joon,"Yoo Jung, Han Yi-seul, James, Shin Yeon-woo",KR,...,1,0,0,14.660949,0.0,0.0,2020,0,05,2020
3,Sonic the Hedgehog,2020-01-01,movie,99.0,1.0,1.0,"Action, Science Fiction, Comedy, Family",Jeff Fowler,"Ben Schwartz, James Marsden, Tika Sumpter, Jim...",US,...,0,0,0,67.267735,0.0,0.0,2020,1,02,2020
4,Demon Slayer -Kimetsu no Yaiba- The Movie: Mug...,2020-01-01,movie,117.0,1.0,1.0,"Animation, Action, Fantasy, Thriller",Haruo Sotozaki,"Natsuki Hanae, Akari Kito, Hiro Shimono, Yoshi...",JP,...,0,0,0,68.296560,0.0,1.0,2020,1,10,2020


In [50]:
df_movie.columns

Index(['title', 'release_date', 'type', 'runtime', 'season_count',
       'episode_count', 'genre', 'director', 'cast', 'country', 'language',
       'production_company', 'synopsis', 'tags', 'imdb_rating',
       'imdb_review_count', 'text_input', 'production_company_is_missing',
       'director_is_missing', 'cast_is_missing', 'imdb_score', 'award_score',
       'top_score', 'release_year', 'success_label', 'month', 'year'],
      dtype='object')

In [51]:
df_movie = df_movie[['title', 'year', 'month', 'type', 'runtime', 'season_count',
       'episode_count', 'genre', 'director', 'cast', 'country', 'language',
       'production_company', 'synopsis', 'tags', 'imdb_rating',
       'imdb_review_count', 'text_input', 'production_company_is_missing',
       'director_is_missing', 'cast_is_missing', 'imdb_score', 'award_score',
       'top_score', 'release_year', 'success_label']]

df_movie.head()

,title,year,month,type,runtime,season_count,episode_count,genre,director,cast,...,imdb_review_count,text_input,production_company_is_missing,director_is_missing,cast_is_missing,imdb_score,award_score,top_score,release_year,success_label
0,Stepmom's Desire,2020,05,movie,78.0,1.0,1.0,"Drama, Romance",Lee Dong-Joon,"Lee Soo, Tae Hee, James, Jung In",...,20,"Sang-jin, who was envious of her neighbor afte...",1,0,0,21.768335,0.0,0.0,2020,1
1,Pizza Dare 1,2020,04,movie,68.0,1.0,1.0,Romance,Kong Ja-kwan,"Yoo Jung, Yoon So-jung, Jin Si-ah, Baek Yoon-j...",...,9,"Seok, who wants to try perverted sex, pesters ...",1,0,0,14.326684,0.0,0.0,2020,0
2,Bosomy Mom,2020,05,movie,69.0,1.0,1.0,"Drama, Romance",Lee Dong-Joon,"Yoo Jung, Han Yi-seul, James, Shin Yeon-woo",...,11,Da-hee gets married to Min-soo who is older th...,1,0,0,14.660949,0.0,0.0,2020,0
3,Sonic the Hedgehog,2020,02,movie,99.0,1.0,1.0,"Action, Science Fiction, Comedy, Family",Jeff Fowler,"Ben Schwartz, James Marsden, Tika Sumpter, Jim...",...,9955,"Powered with incredible speed, Sonic The Hedge...",0,0,0,67.267735,0.0,0.0,2020,1
4,Demon Slayer -Kimetsu no Yaiba- The Movie: Mug...,2020,10,movie,117.0,1.0,1.0,"Animation, Action, Fantasy, Thriller",Haruo Sotozaki,"Natsuki Hanae, Akari Kito, Hiro Shimono, Yoshi...",...,4103,"Tanjiro Kamado, joined with Inosuke Hashibira,...",0,0,0,68.296560,0.0,1.0,2020,1


In [53]:
df_movie.isnull().sum()

title                            0
year                             0
month                            0
type                             0
runtime                          0
season_count                     0
episode_count                    0
genre                            0
director                         0
cast                             0
country                          0
language                         0
production_company               0
synopsis                         0
tags                             0
imdb_rating                      0
imdb_review_count                0
text_input                       0
production_company_is_missing    0
director_is_missing              0
cast_is_missing                  0
imdb_score                       0
award_score                      0
top_score                        0
release_year                     0
success_label                    0
dtype: int64

In [54]:
df_movie.to_csv('./data/train.csv', index=False)

# x_test 만들기

In [ ]:
x_test = pd.read_csv('./data/x_test.csv')
x_test.head()

In [ ]:
x_test.info()

In [ ]:
x_test['release_date'] = pd.to_datetime(x_test['release_date']).dt.year.astype(str)
x_test[x_test['genre'].isna()]
x_test = x_test.dropna(subset=["genre"])
x_test["synopsis"] = x_test["synopsis"].fillna("unknown")

In [ ]:
x_test.info()

In [ ]:
# 안전하게 이름 정규화
def normalize_name(name):
    if not isinstance(name, str):
        return ''
    return name.lower().strip()

# 수상자 이름 세트
oscar_winner_names = set(
    normalize_name(n) for n in df_oscars[df_oscars['Winner']]['Name'].unique()
)

# cast_award_score 계산 (쉼표로 분리 + 예외 처리)
def get_cast_award_score(cast_str):
    if not isinstance(cast_str, str):
        return 0
    cast_names = [normalize_name(name) for name in cast_str.split(',')]
    return sum(name in oscar_winner_names for name in cast_names)

# 적용
x_test['cast_award_score'] = x_test['cast'].apply(get_cast_award_score)
x_test['cast_award_score'] = np.log1p(x_test['cast_award_score'])

In [ ]:
x_test.head()

In [ ]:
# 1. 연도 추출
df_movie['release_year'] = df_movie['release_date'].astype(str).str[:4].astype(int)
df_tmbd_top['release_year'] = df_tmbd_top['release_date'].astype(str).str[:4].astype(int)

# 2. title + release_year 기준으로 병합하여 감독/배우 정보 붙이기
df_people = df_movie[['title', 'release_year', 'director', 'cast']].copy()

df_top_with_people = pd.merge(
    df_tmbd_top,
    df_people,
    how='left',
    on=['title', 'release_year']
)

# 3. 감독별 평균 top_score
director_top_score = (
    df_top_with_people
    .dropna(subset=['director'])
    .groupby('director')['top_score']
    .mean()
    .reset_index()
    .rename(columns={'top_score': 'director_top_score'})
)

# 4. 배우별 평균 top_score
def explode_cast(df):
    df = df.dropna(subset=['cast']).copy()
    df['cast'] = df['cast'].astype(str)
    df['cast_list'] = df['cast'].apply(lambda x: [c.strip() for c in x.split(',')])
    df_exploded = df.explode('cast_list')
    return df_exploded[['cast_list', 'top_score']]

df_cast_exploded = explode_cast(df_top_with_people)

actor_top_score = (
    df_cast_exploded
    .groupby('cast_list')['top_score']
    .mean()
    .reset_index()
    .rename(columns={'cast_list': 'actor', 'top_score': 'actor_top_score'})
)

# 5. dict로 변환 (성능 ↑)
actor_score_dict = dict(zip(actor_top_score['actor'], actor_top_score['actor_top_score']))

def compute_actor_top_score_fast(cast_str):
    if not isinstance(cast_str, str):
        return 0
    cast_list = [actor.strip() for actor in cast_str.split(',')]
    scores = [actor_score_dict.get(actor, 0) for actor in cast_list]
    return np.mean(scores) if scores else 0

# 6. x_test에 감독/배우 점수 병합
x_test = x_test.copy()
x_test = x_test.merge(director_top_score, how='left', on='director')
x_test['director_top_score'] = x_test['director_top_score'].fillna(0)
x_test['actor_top_score'] = x_test['cast'].apply(compute_actor_top_score_fast)


In [ ]:
x_test.head()

In [ ]:
x_test.sort_values(by='cast_award_score', ascending=False)

In [ ]:
# unknown 제외하고 집계
director_all_counts = df_movie.loc[df_movie['director'] != 'unknown', 'director'].value_counts()

df_success = df_movie[(df_movie['success_label'] == 1) & (df_movie['director'] != 'unknown')]
director_success_counts = df_success['director'].value_counts()

# 매핑 시 unknown은 0 또는 별도 값 부여
df_movie['director_productivity_all'] = df_movie['director'].map(director_all_counts).fillna(0)
df_movie['director_productivity_success'] = df_movie['director'].map(director_success_counts).fillna(0)


In [ ]:
df_movie.head()

In [ ]:
# x_test에 매핑, unknown 감독이나 없는 감독은 0으로 채우기
x_test['director_productivity_success'] = x_test['director'].map(director_success_counts).fillna(0)

In [ ]:
x_test.head()

In [ ]:
x_test.sort_values(by='runtime', ascending=False)

In [ ]:
print('df_movie 칼럼: ', df_movie.columns)
print('x_test 칼럼: ', x_test.columns)

In [ ]:
#x_test.to_csv('./data/test.csv', index=False)